In [2]:
import os
import requests
import bs4
from bs4 import BeautifulSoup
import sys
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sb
import numpy as np

In [3]:
print(bs4.__version__)

4.9.3


In [4]:
url = 'https://www.tripadvisor.com.br/Restaurants-g303631-Sao_Paulo_State_of_Sao_Paulo.html'
headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})
resposta=requests.get(url=url,headers=headers)

resposta.encoding='utf-8'



#https://www.tripadvisor.com.br/RestaurantSearch-g303631-oa30-a_date.2021__2D__05__2D__26-a_people.2-a_time.20%3A00%3A00-a_zur.2021__5F__05__5F__26-Sao_Paul.html


In [5]:
print('Código de status da resposta: {}'.format(resposta.status_code))

Código de status da resposta: 200


In [6]:
soup=BeautifulSoup(resposta.text,'html.parser')

In [7]:
tag_restaurantes=soup.find('div',class_="_1kXteagE")

In [8]:
lista_restaurantes=tag_restaurantes.find_all('div',class_="_1llCuDZj")

In [153]:
lista_restaurantes[2]

for palavra in str(lista_restaurantes[6]).split():
    if 'data-test' in palavra:
        palavras2=palavra.split('=')
        p=palavras2[1]
        lista3=p.split('>')
        p2=lista3[0]
        if 'SL' in p2:
            print ('Patrocínio')
        else:
            print ('Restaurante')


Patrocínio


Para cada restaurante:

    -Nome 
    
    -"Estrelas"(0 a 5)
    
    -Avaliações
    
    -Cozinha (steakhouse,italiano,frutos do mar,...)
    
    -Preço 
    
    -Há medidas de segurança? (div class="_1ULcQ4O5 _2TXtEj3w")
    
    -Review
    

In [167]:
lista_nomes=[]
lista_estrelas=[]
lista_avaliacoes=[]
lista_cozinhas=[]
lista_precos=[]
lista_seg=[]
lista_reviews=[]
lista_descricao=[]
for res_html in lista_restaurantes:
    for palavra in str(res_html).split():
        if 'data-test' in palavra:
            palavras2=palavra.split('=')
            p=palavras2[1]
            lista3=p.split('>')
            p2=lista3[0]
            if 'SL' not in p2:
                nome=res_html.find('a',class_='_15_ydu6b').text
                nome=nome.split('.')[1]
                lista_nomes.append (nome)

                estrela=res_html.find ('svg',class_='zWXXYhVR')
                for palavra in str(estrela).split():
                    if 'title' in palavra:
                        palavras=palavra.split('=')
                        nota=palavras[1]
                        nota=nota.replace('"','').replace(',','.')
                        nota=float(nota)
                lista_estrelas.append(nota)

                avaliacao=res_html.find('span',class_='w726Ki5B').text
                avaliacao=int(avaliacao.split()[0].replace('.',''))
                lista_avaliacoes.append(avaliacao)

                descricao=res_html.find('div',class_='MIajtJFg _1cBs8huC _3d9EnJpt')
                lista_descricao.append(descricao)
                cozinha_e_preco=descricao.find_all('span',class_='_1p0FLy4t')
                cozinha=cozinha_e_preco[0].text
                lista_cozinhas.append(cozinha)
                preco=cozinha_e_preco[1].text

                if len(preco)<=5:
                    preco2=len(preco)
                else:
                    lista1=preco.split('-')
                    for i in range(len(lista1)):
                        lista1[i]=lista1[i].replace(' ','')
                        lista1[i]=len(lista1[i])
                    preco2=np.mean(lista1)

                lista_precos.append(preco2)    
            #     if descricao.find('span',class_='_1Cbxp6P3') is not None:
            #         lista_seg.append('Sim')
            #     else:
            #         lista_seg.append('Não')


                reviews=res_html.find_all('div',class_='_3hu2oQ7V')
                reviews=[reviews[0].text,reviews[1].text]
                lista_reviews.append(reviews)
                break

In [168]:
df=pd.DataFrame({'Nome':lista_nomes,
                    'Estrelas':lista_estrelas,
                    'Avaliações':lista_avaliacoes,
                    'Cozinha':lista_cozinhas,
                    'Preço':lista_precos,
                    'Reviews':lista_reviews})

In [169]:
df

,Nome,Estrelas,Avaliações,Cozinha,Preço,Reviews
0,Coco Bambu Conceito Pátio Paulista,5.0,109,Frutos do mar,2.5,"[“Almoço”, “Aniversário dos meus gêmeos ♡”]"
1,Coco Bambu Conceito Vila Olímpia,5.0,563,"Brasileira, Frutos do mar",2.5,"[“Atendimento Kaio Marquês”, “Atendimento Kaio..."
2,HUB Food Art & Lounge,5.0,1021,"Brasileira, Internacional",4.0,"[“Aniversário de casamento”, “Comida impecável!”]"
3,Picanharia Dos Amigos,5.0,1002,"Steakhouse, Brasileira",2.5,"[“Picanharia dos Amigos - Mais que recomendo”,..."
4,Coco Bambu Market Place,4.5,4957,"Brasileira, Bar",2.5,"[“Iremos casar”, “Excelente!”]"
5,Coco Bambu Anália Franco,5.0,12706,"Brasileira, Frutos do mar",2.5,"[“Aniversário”, “Meu aniversário”]"
6,Djapa,4.5,5152,"Japonesa, Frutos do mar",2.5,"[“Bom!”, “Excelente atendimento. Especialmente”]"
7,Coco Bambu Anhembi,5.0,10826,Brasileira,4.0,"[“Almoco”, “Comemoração de aniversario”]"
8,Lhamas Restaurante,5.0,507,"Peruana, Frutos do mar",2.5,"[“Melhor restaurante peruano de São Paula”, “E..."
9,TasteIT Restaurante,5.0,1037,"Bar, Internacional",2.5,"[“Experiência perfeita!”, “Experiência incrível”]"
